# Configuration

In [ ]:
# 모듈 불러오기
from Classes import BirdRecognition, BirdSoundDataset
from torch.utils.data import DataLoader
from pathlib import Path
from datetime import datetime
import torch
import json

# Device 설정
if torch.cuda.is_available(): DEVICE = 'cuda'
else                        : DEVICE = 'cpu'

# 데이터셋과 모델 생성

In [ ]:
with open('./scored_birds.json', 'r') as file:
    CLASSES = json.load(file)

# 훈련/검증 데이터셋
trainset = BirdSoundDataset(Path('./train_metadata_train.csv'), Path('./train_audio'), CLASSES)
train_loader = DataLoader(trainset, batch_size=1, shuffle=True, num_workers=0)

valset = BirdSoundDataset(Path('./train_metadata_validation.csv'), Path('./train_audio'), CLASSES)
val_loader = DataLoader(valset, batch_size=1, shuffle=True, num_workers=0)

# 모델 생성
model = BirdRecognition(CLASSES).to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=0.0000002)
loss_f = torch.nn.BCELoss()

# Annotation 정보 확인하기
* Label 정보 vectorization은 사전에 수행 - labeler.ipynb 파일 참조

In [ ]:
# trainset.annotations.iloc[:3, :]

# 학습

In [ ]:
EPOCH = 200
BEST_MODEL = {'params': 0, 'loss': 10000}

for epoch in range(EPOCH):
    model.train()

    for idx, data in enumerate(train_loader):
        torch.cuda.empty_cache()
        signal = data['signal'].squeeze(0).to(DEVICE)
        label = list(map(float, data['labels'][0][1:-1].split(', ')))
        label = torch.Tensor(label).to(DEVICE)

        out = model(signal)
        loss = loss_f(out, label)

        optim.zero_grad()
        loss.backward()
        optim.step()

        print(f'[EPOCH {epoch+1}] Now traning... {idx+1:05d}/{len(train_loader):05d}', end='\r')

    model.eval()
    val_loss = []

    for idx, data in enumerate(val_loader):
        torch.cuda.empty_cache()
        signal = data['signal'].squeeze(0).to(DEVICE)
        label = list(map(float, data['labels'][0][1:-1].split(', ')))
        label = torch.Tensor(label).to(DEVICE)

        out = model(signal)
        loss = loss_f(out, label)
        val_loss.append(loss.detach().to('cpu'))
        print(f'[EPOCH {epoch+1}] Validating model... {idx+1:05d}/{len(val_loader):05d}{" ":30s}', end='\r')
    
    val_loss = sum(val_loss) / len(val_loss)
    print(f'\nValidation loss: {val_loss:6.4f}')

    if BEST_MODEL['loss'] > val_loss:
        print('It is the best model. Saving the parameters.')
        BEST_MODEL['loss'] = val_loss
        BEST_MODEL['params'] = model.state_dict()

# 최고 성능 파라미터 파일로 저장

In [ ]:
from datetime import datetime
import pickle

for key in BEST_MODEL['params'].keys():
    BEST_MODEL['params'][key] = BEST_MODEL['params'][key].to('cpu')

param_dir = Path('./params')
param_dir.mkdir(exist_ok=True)

time = datetime.now()
with open(param_dir / f'{time.year:04d}{time.month:02d}{time.day:02d}{time.hour:02d}{time.minute:02d}{time.second:02d}.params', 'wb') as file:
    pickle.dump(BEST_MODEL['params'], file)